In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import Sequential
import datetime



In [3]:
data_train = pd.read_csv("train_dataset_train.csv")

In [4]:
data_train.head(3)

,id,Easting,Northing,Height,Reflectance,Class
0,2321251,431696.5375,6.032319e+06,69.2226,-11.14,0
1,3515173,431710.3835,6.032291e+06,68.9711,-15.16,3
2,2320295,431696.8099,6.032322e+06,69.2453,-13.59,0


In [5]:
data_y = data_train["Class"]
encod_y = LabelBinarizer()
Y_train = encod_y.fit_transform(data_y)

In [45]:
data_train.

,id,Easting,Northing,Height,Reflectance,Class
count,4222094,4222094,4222094,4222094,4222094,4222094
unique,1,1,1,1,1,1
top,False,False,False,False,False,False
freq,4222094,4222094,4222094,4222094,4222094,4222094


In [55]:
from sklearn.cross_decomposition import CCA
def get_train_dataset_x(data_x):

  data_X = data_x.copy()
  norm = MinMaxScaler()
  pca = PCA(5)
  res = norm.fit_transform(data_X)
  res = pca.fit_transform(res)
  X_train = res

  return X_train, norm, pca

In [52]:
def get_test_dataset_x(data_x, norm, pca):

  data_X = data_x.copy()
  res = norm.transform(data_X)
  res = pca.transform(res)
  X_test = res

  return X_test

In [56]:
data_x = data_train[data_train.columns[:-1]]
X_train, norm, pca = get_train_dataset_x(data_x)
X_train

array([[-6.87777706e-02,  1.36628784e-01, -1.95210048e-04,
         2.51397894e-02,  3.63875895e-04],
       [ 3.38757391e-02, -1.18561603e-01,  1.27019441e-01,
         1.32360504e-02, -2.29855115e-02],
       [-6.39148418e-02,  1.46737772e-01,  3.58513554e-02,
         7.32512653e-02,  1.86958274e-03],
       ...,
       [ 3.34267475e-01, -3.49080135e-01, -1.53554982e-02,
         9.78825008e-03,  3.97604025e-04],
       [ 1.53358403e-01,  1.40338784e-02, -1.69184206e-02,
         3.28694075e-03,  2.65370578e-03],
       [-3.65760042e-01,  2.80016386e-01, -2.41076829e-01,
         8.97094987e-02,  1.10326111e-02]])

In [57]:
X_train, norm, pca = get_train_dataset_x(data_x)

#Создание модели обучения

In [10]:
import tensorflow as tf
import gc
mods = []
hists = []
num = 0
metric = tf.keras.metrics.Recall()

dots = [False, True]
acts = ["relu", tf.keras.activations.linear, "sigmoid"]
ends = ["softmax"]
ks = list(range(5, 101))
maximum = len(dots)*len(acts)*len(ends)*len(ks)

for dot in dots:
    for act in acts:
        for end in ends:
            for k in ks:
                model = Sequential()
                model.add(Dense(k, activation=act, input_shape = (X_train.shape[1],)))
                if dot:
                    model.add(Dense(k//2, activation=act))
                else:
                    model.add(Dense(k, activation=act))
                model.add(Dense(Y_train.shape[1], activation=end))
                model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=[metric])

                history = model.fit(X_train, Y_train, epochs=10, validation_split = 0.3, batch_size=10000, verbose = 0)

                print("\r",dot, act, end, k, end = " ")
                num += 1
                print(f" |  {int(num/maximum*100)} % ","[","="*int(num/maximum*100), "-"*int(100-num/maximum*100),"]", sep="", end="")

                mods.append(model)
                del model
                hists.append(history)
                del history
                gc.collect()

 True sigmoid softmax 100  |  100 % [====================================================================================================]=============----------------]]

In [20]:
A = np.array([i.history["val_loss"][-1] for i in hists])
B = np.array([i.history["val_recall_1"][-1] for i in hists])
minimum = [A.tolist().index(i) for i in sorted(A)]
maximum = [B.tolist().index(i) for i in sorted(B, reverse=True)]
m = [f"{k} {end} {act} {dot}" for dot in dots for act in acts for end in ends for k in ks]
list(zip([m[i] for i in minimum][:10], [m[i] for i in maximum][:10]))

[('98 softmax relu False', '94 softmax relu True'),
 ('89 softmax relu False', '94 softmax relu False'),
 ('75 softmax relu False', '89 softmax relu False'),
 ('100 softmax relu False', '71 softmax relu False'),
 ('91 softmax relu False', '98 softmax relu False'),
 ('92 softmax relu False', '73 softmax relu False'),
 ('94 softmax relu False', '92 softmax relu False'),
 ('94 softmax relu True', '84 softmax relu False'),
 ('93 softmax relu False', '91 softmax relu False'),
 ('71 softmax relu False', '83 softmax relu False')]

In [10]:
import tensorflow as tf
metric = tf.keras.metrics.Recall()

In [58]:
model_best = Sequential()
model_best.add(Dense(1000, activation="relu", input_shape = (X_train.shape[1],)))
model_best.add(Dense(1000, activation="relu"))
model_best.add(Dense(Y_train.shape[1], activation="softmax"))
model_best.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=[metric])

In [79]:
history_best = model_best.fit(X_train, Y_train, epochs=1, validation_split = 0.2, batch_size=40000, verbose = 1)

85/85 [==============================] - 22s 257ms/step - loss: 0.0325 - recall: 0.9860 - val_loss: 0.0381 - val_recall: 0.9843

In [81]:
test = pd.read_csv("test_dataset_test.csv")
X_test = get_test_dataset_x(test[test.columns[:]], norm, pca)
Y_test = model_best.predict(X_test, batch_size = 100000, verbose=1)

19/19 [==============================] - 4s 229ms/step


In [82]:
classif = encod_y.inverse_transform(Y_test)
id = test["id"].to_numpy()
res = list(zip(id,classif))
out = pd.DataFrame(res, columns=["id", "Class"])
out.to_csv(f"result_9_1200.csv", index=False)

In [83]:
model_best.save(f"model_9_1200.h5")

In [110]:
best_model = tf.keras.models.load_model("model_9_1147.h5")
best_model.optimizer = tf.keras.optimizers.Adamax()

In [ ]:
history_best = best_model.fit(X_train, Y_train, epochs=20, validation_split = 0.2, batch_size=40000, verbose = 1)





Epoch 1/20
85/85 [==============================] - 22s 256ms/step - loss: 0.0303 - recall: 0.9870 - val_loss: 0.0376 - val_recall: 0.9847